<a href="https://colab.research.google.com/github/Prach1/CS69002_9A_18CS60R30/blob/master/DL_Assign9a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import pandas as pd
import numpy as np
import io
import re

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Train_20K.csv to Train_20K.csv


In [4]:
df = pd.read_csv(io.StringIO(uploaded['Train_20K.csv'].decode('utf-8')), sep='\t')
print(df.head())

                                                text  label
0  John Waters has given us a genuinely enjoyable...      1
1  This first two seasons of this comedy series w...      1
2  What an unfortunate mess is "Shiner." I wanted...      0
3  I'm not entirely sure Rob Schmidt qualifies as...      1
4  i wasn't sure whether to laugh or cry. Porrett...      0


In [0]:
for chunk in df:
  print(chunk)

text
label


In [5]:
# for i, j in df.iterrows(): 
#     print(i)
#     print(j)

print(type(df['text']))
print(type(df['label']))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [42]:
text_reviews = df['text'].astype(str).tolist()
text_labels = df['label'].astype(int)
print(type(text_labels))

text_reviews = [x.lower() for x in text_reviews]
#print(type(text_reviews))
#print(text_reviews[0])

<class 'pandas.core.series.Series'>


In [0]:
import re

def preprocess(text):    
  text = re.sub(r'<br.*?>',' ',text)
  text = re.sub(r'[^\w\s]',' ',text) 
  text = re.sub(r'[0-9]+', ' ', text)
  
  return text
  

In [30]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

train_data = []
train_data_X = []
stop = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

for review in text_reviews:
  new_review = preprocess(review)
  train_data.append(new_review)
  
print(train_data[0])

for review in train_data:
  review = [lemmatizer.lemmatize(word) for word in review.split() if word not in stop and word != '' and len(lemmatizer.lemmatize(word)) > 1]
  train_data_X.append(review)
  
print(train_data_X[0])

john waters has given us a genuinely enjoyable film  this certainly isn t without its shocking waters esque moments  but it is tamer than his older culty stuff  such as  pink flamingoes    pecker  harkens back to john s early mainstream stage in that it reminds the viewer of the same kind of humor that was evident in  polyester   overall  a really fun comedy with some great moments 
['john', 'water', 'given', 'genuinely', 'enjoyable', 'film', 'certainly', 'without', 'shocking', 'water', 'esque', 'moment', 'tamer', 'older', 'culty', 'stuff', 'pink', 'flamingo', 'pecker', 'harkens', 'back', 'john', 'early', 'mainstream', 'stage', 'reminds', 'viewer', 'kind', 'humor', 'evident', 'polyester', 'overall', 'really', 'fun', 'comedy', 'great', 'moment']


In [0]:
def generate_word_ids(dataset):
  word_to_ix = {}
 
  for sent in dataset:
      for word in sent:
          if word not in word_to_ix:
              word_to_ix[word] = len(word_to_ix)
              
  return word_to_ix


In [0]:
word_to_ix = generate_word_ids(train_data_X)
#print(word_to_ix)
VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

In [0]:
def make_bow_vector(sentence):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        if word not in word_to_ix:
            raise ValueError('Word',word,' not present in the dictionary. Sorry!')
        else:
            vec[word_to_ix[word]]+=1
    return vec / len(word_to_ix)

def make_target(label, label_to_ix):
    #return torch.LongTensor([label_to_ix[label]])
    return torch.LongTensor(label)

In [0]:
train = []

for review in train_data_X:
  embedding = torch.autograd.Variable(make_bow_vector(review))
  train.append(embedding)

In [46]:
print(len(train[0]))

57543


In [47]:
data = []

for index,review in enumerate(train):
  data.append((review,text_labels[index]))
  
print(data[0])

(tensor([3.4757e-05, 3.4757e-05, 1.7378e-05,  ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00]), 1)


In [58]:
label_to_ix = {1: 0, 0: 1}
ix_to_label = {v: k for k, v in label_to_ix.items()}

label_to_ix, ix_to_label

({0: 1, 1: 0}, {0: 1, 1: 0})

In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [0]:
class BOWClassifier(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(BOWClassifier, self).__init__()
    self.lin = nn.Linear(vocab_size, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin(x))

In [53]:
bow = BOWClassifier(NUM_LABELS, VOCAB_SIZE)
print(bow)

BOWClassifier(
  (lin): Linear(in_features=57543, out_features=2, bias=True)
)


In [51]:
for param in bow.parameters():
    print(param,param.size())

Parameter containing:
tensor([[ 3.1872e-03,  3.4601e-03, -9.7662e-04,  ..., -3.5356e-03,
         -3.2626e-03, -1.9946e-03],
        [-1.0365e-03, -5.0110e-05, -6.6238e-04,  ...,  1.5359e-03,
         -1.2205e-03,  8.8897e-04]], requires_grad=True) torch.Size([2, 57543])
Parameter containing:
tensor([ 0.0032, -0.0020], requires_grad=True) torch.Size([2])


In [57]:
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

print(label_to_ix)

{'1': 0, '0': 1}


In [67]:
# the training loop
for epoch in range(10):
    for instance, label in data:
        bow.zero_grad()
        bow_vec = Variable(instance)
        label = Variable(make_target(label, label_to_ix))
        probs = bow(bow_vec) # forward pass
        print(probs)
        print(label)
        
        if(probs[0] > probs[1]):
          probs = probs[0]
        else:
          probs = probs[1]
        
        print(probs)
        print(label)
        loss = loss_function(probs, label)
        loss.backward()
        print('CURRENT LOSS: {}'.format(loss.data))
        opt.step()

tensor([0.5003, 0.4997], grad_fn=<SoftmaxBackward>)
tensor([120318296])
tensor(0.5003, grad_fn=<SelectBackward>)
tensor([120318296])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


ValueError: ignored